第一种：不带参数的层

In [1]:
import torch
import torch.nn.functional as F
from torch import  nn

In [2]:
class CenterLyear(nn.Module):   # 定义一个数据中心化的层
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [3]:
layer = CenterLyear()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [4]:
net = nn.Sequential(nn.Linear(8, 128), CenterLyear())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

第二种：带参数的层，例如定义自己的线性层

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))    # weight和bias都要设置为nn.Parameter
        self.bias = nn.Parameter(torch.zeros(units))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)   # 和模型自带的线性层不同我们这里集成了RELU层

In [6]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.3818, -0.3641,  0.1131],
        [ 0.4957, -0.7303, -1.2564],
        [-0.9770,  1.5210, -0.2411],
        [-2.9365,  0.8603,  1.4928],
        [-0.1687,  0.4447, -1.1634]], requires_grad=True)

In [26]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.7320, 0.0000],
        [0.0000, 0.7384, 0.0000]])

In [28]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[13.2326],
        [15.0390]])